In [4]:
from rank_bm25 import BM25Okapi

corpus = [
    "Hello there good man!",
    "It is quite windy in London",
    "How is the weather today?"
]

tokenized_corpus = [doc.split(" ") for doc in corpus]

bm25 = BM25Okapi(tokenized_corpus)

In [10]:
query = "windy London"
tokenized_query = query.split(" ")

doc_scores = bm25.get_scores(tokenized_query)
doc_scores

array([0.        , 0.93729472, 0.        ])

In [12]:
import numpy as np
np.argsort(-doc_scores)

array([1, 0, 2])

In [13]:
doc_scores

array([0.        , 0.93729472, 0.        ])

In [9]:
bm25.get_top_n(tokenized_query, corpus, n=1)

['It is quite windy in London']

In [17]:
import json
import numpy as np

from rank_bm25 import BM25Okapi

class BM25Retrieval:
  def __init__(self, tokenize_fn, data_path):
    """
    bm25 모듈 생성
    """
    with open(data_path, "r") as json_data:
      wiki_data = json.load(json_data)
    
    wiki_data_keys = list(wiki_data.keys())
    self.contexts = [
        wiki_data[key]["text"] for key in wiki_data_keys
    ]  # 여기 keys값의 text값을 받도록하기
    self.tokenize_fn = tokenize_fn
    self.__get_sparse_embedding()
    
  def __get_sparse_embedding(self):
      """
      주어진 passage를 fit 해준다.
      """
      tokenized_contexts = list(map(self.tokenize_fn, self.contexts))
      self.bm25 = BM25Okapi(tokenized_contexts)
  
  def get_relevant_doc(self, query, k=1):
      """
      질문을 입력받고, k 만큼 랭킹을 가져온다.
      """
      tokenized_query = self.tokenize_fn(query)
      raw_doc_scores = self.bm25.get_scores(tokenized_query)
      
      doc_scores_index_desc = np.argsort(-raw_doc_scores)
      doc_scores = raw_doc_scores[doc_scores_index_desc]
      
      doc_list = self.bm25.get_top_n(tokenized_query, self.contexts, k)
      
      return doc_scores[:k], doc_list


In [18]:
tokenize_fn = lambda x: x.split()
retrieval = BM25Retrieval(tokenize_fn=tokenize_fn, data_path="../../data/wikipedia_documents.json")

In [19]:
query = "미국의 대통령은 누구인가?"
doc_scores, doc_list = retrieval.get_relevant_doc(query, 3)

In [20]:
doc_scores

array([14.96697037, 13.71463582, 12.38848409])

In [21]:
doc_list

['대통령 지시(Presidential directive)는 미국 대통령의 국가안보 관련 행정명령이다. NSC의 조언과 동의(Advice and consent)가 필요하다. 국가안보 대통령 지시가 보다 뜻이 명확하다.\n\n역대 대통령들은 다양한 용어를 사용했다. 케네디 대통령은 국가안보실행메모(NSAM, National Security Action Memorandums), 닉슨과 포드 대통령은 국가안보결정메모(NSDM, National Security Decision Memorandums), 클린턴 대통령은 대통령결정지시, 조지 부시 대통령은 국가안보대통령지시, 오바마 대통령은 대통령정책지시라고 부른다.\n\n미국의 대통령 지시는 비밀명령으로 내려지기도 하는데, 이는 수십년이 지나면 비밀해제되어 일반에 공개된다. 미국 육군 정보와 보안 사령부와 같이, 군대에 특정 부대를 창설하는 경우, 대통령 지시만으로 창설되곤 한다.',
 '한미 관계\n\n이명박 대통령은 이미 여러 차례 “새로운 정부에서는 한미관계를 더욱 강화하겠다”는 뜻을 밝혔고 실제로 한미관계는 더욱 강화될 것이라 보는 의견이 많았다.  이명박 대통령은 한미동맹 강화를 위해서는 미국 주도의 대량살상무기 확산방지구상(PSI)과 MD 계획에도 적극 참여해야 한다는 입장인 것으로 알려졌다. \n\n미국산 쇠고기 수입 협상 논란은 촛불집회로 비화되어 2008년 미국산 쇠고기 수입 협상 논란을 경험했다. 이때 이명박 정부는 미국과의 동맹강화를 위해 이러한 논란을 묵살하는 정책을 폈으며, 결과적으로 성공적으로 사태를 진압하여 한미관계 강화를 향한 일관된 원칙을 증명했다.\n\n이후, 2009년 미국의 새로운 오바마 행정부의 등장으로 외교관계는 일시적으로 정체되었다가, 현재는 정상회담을 통해 북핵문제에 대한 공조 및 G20 정상 회의 등에 대한 포괄적 합의를 하는 데에 이르렀다. \n\n2009년 11월 19일에는 미국의 오바마 대통령이 방한했다. 여기서 이명박 한국 대통령과 오바마 미국 대통령은 한미동맹 강화 

In [1]:
from datasets import load_from_disk

datasets = load_from_disk("../../data/test_dataset")

In [2]:
datasets

DatasetDict({
    validation: Dataset({
        features: ['question', 'id'],
        num_rows: 600
    })
})